In [12]:
import pandas as pd

In [13]:
import numpy as np
import re
df = pd.read_csv("../data/raw/Pattern Seekers Interview (Responses) - Form Responses 1.csv")
df.drop(columns=["Timestamp"], inplace=True)
# df.drop(columns=["How is the cow?"], inplace=True)

df['Which group is the best in Cluster 11?'] = df['Which group is the best in Cluster 11?'].replace({'Pattern Seekers': 5, 'Group 5': 5})

def clean_sleep(val):
    if isinstance(val, str):
        if "hour" in val:
            match = re.match(r"(\d+)\s*hours?\s*(\d*)\s*minutes?", val)
            if match:
                hours = int(match.group(1))
                minutes = int(match.group(2)) if match.group(2) else 0
                return hours + minutes / 60
        if val in ["cake", "A HUNEDDDDDDDDD", "6 or 7", "54 hours"]:
            return np.nan
    try:
        val_float = float(val)
        if val_float in [17, -3, 54]:
            return np.nan
        return val_float
    except:
        return np.nan

# Clean the column, but don't drop NaNs yet
df['How many hours of sleep did you get yesterday?'] = df['How many hours of sleep did you get yesterday?'].apply(clean_sleep)

# Impute NaNs with the mean of the valid values
mean_sleep = df['How many hours of sleep did you get yesterday?'].mean()
df['How many hours of sleep did you get yesterday?'].fillna(mean_sleep, inplace=True)

df.reset_index(drop=True, inplace=True)
df.head(17)

/var/folders/70/nplvvvvs69dcsdrp16h2_0l80000gn/T/ipykernel_73144/2910562590.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Which group is the best in Cluster 11?'] = df['Which group is the best in Cluster 11?'].replace({'Pattern Seekers': 5, 'Group 5': 5})
/var/folders/70/nplvvvvs69dcsdrp16h2_0l80000gn/T/ipykernel_73144/2910562590.py:32: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col]

,What is your name?,How many hours of sleep did you get yesterday?,Rate your roommate on a scale of 1-10,How good is the food?,How is the cow?,Which group is the best in Cluster 11?
0,2,5.466,5,2,like beef,5
1,2,8.000,10,4,I like the milk,5
2,2,7.000,10,3,good,5
3,2,1.000,8,3,moo,5
4,3,6.000,8,4,good,5
5,2,6.150,10,1,Moo,5
6,3,5.466,8,3,moo,5
7,2,2.000,10,5,cheeseburger,5
8,2,8.000,10,3,it wants to eat you,5
9,3,9.000,10,3,smelly,5


In [14]:
!pip install vaderSentiment
# COW SENTIMENT ANALYSIS

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

# Apply sentiment analysis to the "How is the cow?" column
df['Cow Sentiment'] = df['How is the cow?'].astype(str).apply(lambda x: analyzer.polarity_scores(x)['compound'])

df.drop(columns=["How is the cow?"], inplace=True)

df.head()

df.to_csv("../data/cleaned/cleaned.csv")